# 🧠 Mongochain: MongoDB Atlas as an Agent Memory Layer

This notebook demonstrates how **MongoDB Atlas** can serve as a powerful memory layer for AI agents with **user-specific memories**.

**What you'll learn:**
- Creating agents with personas backed by MongoDB
- User-specific memories (conversation, short-term, long-term)
- Automatic fact extraction and storage
- Changing agent personalities dynamically
- Multi-agent collaboration through shared memories

**Prerequisites:**
- MongoDB Atlas cluster (free tier works!)
- Voyage AI API key (for embeddings)
- OpenAI API key (or Anthropic/Google)

---
## Step 1: Install Dependencies

In [ ]:
!pip install -q pymongo voyageai openai anthropic google-generativeai

## Step 2: Install Mongochain

In [ ]:
!pip install -q git+https://github.com/robinvarghese/mongochain.git

---
## Step 3: Configure API Keys

**Use Colab Secrets (Recommended):**
1. Click the 🔑 key icon in the left sidebar
2. Add: `MONGO_URI`, `VOYAGE_API_KEY`, `OPENAI_API_KEY`

In [ ]:
try:
    from google.colab import userdata
    MONGO_URI = userdata.get('MONGO_URI')
    VOYAGE_API_KEY = userdata.get('VOYAGE_API_KEY')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ Loaded API keys from Colab secrets")
except:
    # Fallback: enter your keys directly
    MONGO_URI = "mongodb+srv://<username>:<password>@<cluster>.mongodb.net/"
    VOYAGE_API_KEY = "voy-..."
    OPENAI_API_KEY = "sk-..."
    print("⚠️ Using hardcoded keys")

## Step 4: Verify MongoDB Connection

In [ ]:
from pymongo import MongoClient

try:
    client = MongoClient(MONGO_URI)
    client.admin.command('ping')
    print("✅ Successfully connected to MongoDB Atlas!")
    client.close()
except Exception as e:
    print(f"❌ Connection failed: {e}")

## Step 5: Set Up Test User

All memories are user-specific. Let's define our test user.

In [ ]:
# Define the user for this demo
USER_EMAIL = "demo@example.com"
print(f"📧 Test user: {USER_EMAIL}")

---
## 🤖 Demo 1: Create an Agent

Each agent gets its own MongoDB database with three collections:
- `conversation_history` - Raw messages with embeddings (90-day TTL)
- `short_term_memory` - Session summaries (7-day TTL)
- `long_term_memory` - Persistent user facts (no TTL)

In [ ]:
from mongochain import MongoAgent

alice = MongoAgent(
    name="alice",
    persona="""You are Alice, a friendly and knowledgeable assistant.
    You remember details about users and personalize your responses.
    You're helpful, encouraging, and give clear explanations.""",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=OPENAI_API_KEY,
    llm_provider="openai"
)

---
## 💬 Demo 2: Chat with User Context

Notice: `chat()` now requires a `user_id` (email). The agent will:
1. Store the conversation with embeddings
2. Auto-extract user facts to long-term memory
3. Use past context to personalize responses

In [ ]:
# First message - introduce yourself!
response = alice.chat(
    user_id=USER_EMAIL,
    message="Hi Alice! I'm a DevOps engineer at TechCorp. I mainly work with MongoDB and Kubernetes."
)
print("🤖 Alice:", response)

In [ ]:
# Continue the conversation
response = alice.chat(
    user_id=USER_EMAIL,
    message="I'm trying to optimize our MongoDB queries. We have about 50 million documents."
)
print("🤖 Alice:", response)

In [ ]:
# Ask a follow-up - Alice should remember context
response = alice.chat(
    user_id=USER_EMAIL,
    message="What indexing strategies would you recommend for my use case?"
)
print("🤖 Alice:", response)

### 🧠 Check What Alice Learned

The agent automatically extracts facts about the user and stores them in long-term memory!

In [ ]:
# See what Alice learned about the user
memories = alice.get_user_memories(USER_EMAIL)

print(f"🧠 Alice's Long-Term Memories about {USER_EMAIL}:")
print("=" * 50)
for m in memories:
    print(f"  [{m['category']}] {m['content']}")

In [ ]:
# Check memory stats for this user
stats = alice.get_user_stats(USER_EMAIL)

print(f"\n📊 Memory Stats for {USER_EMAIL}:")
print(f"  Conversation messages: {stats['conversation_count']}")
print(f"  Short-term summaries:  {stats['short_term_count']}")
print(f"  Long-term facts:       {stats['long_term_count']}")

### 📝 Manually Store a Memory

You can also explicitly store facts about users.

In [ ]:
# Manually store a user preference
alice.store_user_memory(
    user_id=USER_EMAIL,
    content="User prefers code examples in Python",
    category="preference"
)

alice.store_user_memory(
    user_id=USER_EMAIL,
    content="User's MongoDB cluster has 3 shards with 64GB RAM each",
    category="infrastructure"
)

In [ ]:
# Verify the memories were stored
memories = alice.get_user_memories(USER_EMAIL)
print(f"Total memories: {len(memories)}")
for m in memories:
    print(f"  [{m['category']}] {m['content']}")

---
## 🎭 Demo 3: Dynamic Persona Changes

Change Alice's personality and see how responses differ!

In [ ]:
# Current persona: Friendly assistant
print("🎭 PERSONA: Friendly Assistant")
print("-" * 40)
response = alice.chat(USER_EMAIL, "Explain database sharding in simple terms.")
print("🤖 Alice:", response)

In [ ]:
# Change to grumpy professor
alice.set_persona(
    """You are Professor Alice, a brilliant but grumpy database expert.
    You give accurate but very brief, slightly impatient answers.
    You sigh and make sarcastic comments about 'obvious' questions."""
)

print("\n🎭 PERSONA: Grumpy Professor")
print("-" * 40)
response = alice.chat(USER_EMAIL, "Explain database sharding in simple terms.")
print("🤖 Professor Alice:", response)

In [ ]:
# Change to pirate
alice.set_persona(
    """You are Captain Alice, a pirate who became a database expert.
    You explain everything using nautical metaphors and pirate speak.
    You say 'Arrr!', 'ye scurvy dog', 'shiver me timbers', etc."""
)

print("\n🎭 PERSONA: Pirate Captain")
print("-" * 40)
response = alice.chat(USER_EMAIL, "Explain database sharding in simple terms.")
print("🏴‍☠️ Captain Alice:", response)

In [ ]:
# Reset to normal
alice.set_persona(
    """You are Alice, a knowledgeable and helpful database specialist.
    You give clear, practical advice based on what you know about the user."""
)
print("✅ Persona reset to normal")

---
## 👥 Demo 4: Multi-Agent Collaboration

Create Bob, who can access Alice's memories about users.

In [ ]:
bob = MongoAgent(
    name="bob",
    persona="""You are Bob, a DevOps optimization specialist.
    You give practical, actionable recommendations.
    You can access shared knowledge from other agents.""",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=OPENAI_API_KEY,
    collaborators=["alice"]  # Bob can read Alice's user memories!
)

In [ ]:
# Bob asks about the user - he'll access Alice's memories!
response = bob.chat(
    user_id=USER_EMAIL,
    message="What do you know about my infrastructure? Can you suggest optimizations?"
)
print("🤖 Bob:", response)

In [ ]:
# Bob gives specific recommendations based on shared knowledge
response = bob.chat(
    user_id=USER_EMAIL,
    message="Given my MongoDB setup, what monitoring should I implement?"
)
print("🤖 Bob:", response)

---
## 🔍 Demo 5: Inspect MongoDB Directly

Let's look at what's actually stored in MongoDB.

In [ ]:
from pymongo import MongoClient
client = MongoClient(MONGO_URI)

# List agent databases
print("📁 Agent Databases:")
for db_name in client.list_database_names():
    if db_name not in ['admin', 'local', 'config']:
        db = client[db_name]
        if 'conversation_history' in db.list_collection_names():
            print(f"  🤖 {db_name}")
            for coll in db.list_collection_names():
                count = db[coll].count_documents({})
                print(f"      └── {coll}: {count} docs")

In [ ]:
# Look at conversation history (with embeddings!)
alice_db = client['alice']

print(f"💬 Conversations for {USER_EMAIL}:")
print("=" * 50)
for msg in alice_db.conversation_history.find(
    {"user_id": USER_EMAIL},
    {"embedding": 0}  # Hide embedding for readability
).sort("timestamp", -1).limit(5):
    role = "👤" if msg['role'] == 'user' else "🤖"
    content = msg['content'][:80].replace('\n', ' ')
    print(f"{role} {content}...")

In [ ]:
# Look at long-term memories
print(f"\n🧠 Long-Term Memories for {USER_EMAIL}:")
print("=" * 50)
for mem in alice_db.long_term_memory.find(
    {"user_id": USER_EMAIL},
    {"embedding": 0}
):
    print(f"  [{mem['category']}] {mem['content']}")
    print(f"    Source: {mem.get('metadata', {}).get('source', 'unknown')}")

In [ ]:
# Show that embeddings are stored
sample = alice_db.conversation_history.find_one({"user_id": USER_EMAIL})
if sample and 'embedding' in sample:
    print("\n🔢 Embedding Sample:")
    print(f"  Dimensions: {len(sample['embedding'])}")
    print(f"  First 5 values: {sample['embedding'][:5]}")
    print("  → These enable semantic search!")

---
## 📋 Demo 6: Session Management

End a session to save a summary to short-term memory.

In [ ]:
# End the session - this saves a summary
alice.end_session(USER_EMAIL)

In [ ]:
# Check short-term summaries
print(f"📝 Short-Term Summaries for {USER_EMAIL}:")
for summary in alice_db.short_term_memory.find(
    {"user_id": USER_EMAIL},
    {"embedding": 0}
):
    print(f"  Summary: {summary['summary']}")
    print(f"  Topics: {summary['topics_discussed']}")
    print(f"  Actions: {summary['actions_taken']}")

---
## 🧹 Cleanup (Optional)

In [ ]:
# Clear memories for this user (uncomment to run)
# alice.clear_user_memories(USER_EMAIL)
# bob.clear_user_memories(USER_EMAIL)

# Or drop entire databases (uncomment to run)
# client.drop_database('alice')
# client.drop_database('bob')

print("💡 Uncomment lines above to clear data")

In [ ]:
client.close()
print("✅ Demo complete!")

---
## 📚 Summary

### Memory Architecture

| Collection | Purpose | TTL | User-Specific |
|------------|---------|-----|---------------|
| `conversation_history` | Raw chat + embeddings | 90 days | ✅ |
| `short_term_memory` | Session summaries | 7 days | ✅ |
| `long_term_memory` | User facts | Never | ✅ |

### Key Features

1. **User-Specific Memories** - All memories tied to `user_id` (email)
2. **Auto-Extraction** - LLM automatically extracts user facts
3. **Vector Search** - Semantic search across all memory types
4. **TTL Indexes** - Automatic cleanup of old data
5. **Multi-Agent** - Agents can share user knowledge

### MongoDB Features Demonstrated

- **Atlas Vector Search** - Semantic memory retrieval
- **TTL Indexes** - Automatic data expiration
- **Compound Indexes** - Efficient user + time queries
- **Document Model** - Flexible schema per memory type